In [62]:
import pandas as pd
import scipy as sp
import matplotlib as plt
import numpy as np
import json
from random import shuffle, random
from sklearn.cross_validation import ShuffleSplit, train_test_split

In [66]:
def HaversineDistance(c1, c2): 
  lon_diff = np.abs(c1[0]-c2[0])*np.pi/360.0
  lat_diff = np.abs(c1[1]-c2[1])*np.pi/360.0
  a = np.sin(lat_diff)**2 + np.cos(c1[1]*np.pi/180.0) * np.cos(c2[1]*np.pi/180.0) * np.sin(lon_diff)**2
  d = 2*6371*np.arctan2(np.sqrt(a), np.sqrt(1-a))
  return d


def load_data(num_records_to_load=10):
    submission_df = pd.read_csv('/home/tony/ML/taxi/taxi2_time/test.csv')
    submission_df['POLYLINE'] = submission_df['POLYLINE'].apply(json.loads)
    submission_df['COORDS_LEN'] = submission_df['POLYLINE'].apply(len)
    submission_df['START'] = submission_df['POLYLINE'].apply(lambda x: x[0])

    # read train
    taxi_df = pd.read_csv('/home/tony/ML/taxi/taxi2_time/train.csv', nrows=num_records_to_load)
    taxi_df['POLYLINE'] = taxi_df['POLYLINE'].apply(json.loads)
    taxi_df['COORDS_LEN'] = taxi_df['POLYLINE'].apply(len)
    taxi_df = taxi_df[taxi_df.COORDS_LEN > 10]
    taxi_df['START'] = taxi_df['POLYLINE'].apply(lambda x: x[0])
    taxi_df['END'] = taxi_df['POLYLINE'].apply(lambda x: x[-1])
    
    return taxi_df, submission_df


def createTrainTestSplit(df, percent_test=0.1):
    num_rows = len(df)
    num_train = num_rows - int(num_rows*percent_test)
    mask = np.random.rand(num_rows) > percent_test
    
    train_df, test_df = df[mask], df[~mask]
    train_time, test_time = 15*train_df['COORDS_LEN'].values, 15*test_df['COORDS_LEN'].values
    train_end, test_end = train_df['END'].values, test_df['END'].values

    # Save reference to complete path for analytical purposes
    test_df['POLYLINE_ACTUAL'] = test_df['POLYLINE'].values[:]
    
    #Create partial paths for the test data
    #coords = test_df['POLYLINE'].values[:]
    #partial_lengths = [round(0.5*len(coord)) for coord in coords]
    #test_df['POLYLINE'] = [coords[:n] for n in partial_lengths]
    test_df['POLYLINE'] = [coord[:int(round(0.5*len(coord)))] for coord in test_df['POLYLINE'].values]
    
    
    #Drop all data that we shouldn't have during training
    test_df = test_df.drop(['COORDS_LEN', 'END'], axis=1)
    test_df['COORDS_LEN'] = test_df['POLYLINE'].apply(len)
    
    return train_df, test_df, train_time, test_time, train_end, test_end

def travelTimeScore(pred_times, actual_times):
    score = np.sqrt(np.mean((np.log(pred_times+1)-np.log(actual_times+1))**2))
    return score

def travelEndScore(pred_ends, actual_ends):
    num_points = len(pred_ends)
    preds, actuals = pred_ends.values, actual_ends.values
    score = np.mean([HaversineDistance(preds[i], actuals[i]) for i in range(num_points)])
    return score
    
def submitTravelTime(validation_df, filename):
    validation_df[['TRIP_ID', 'TRAVEL_TIME']].to_csv(filename, index=False)
    
def submitTravelDestination(validation_df, filename):
    validation_df[['TRIP_ID', 'LATITUDE', 'LONGITUDE']].to_csv(filename, index=False)

In [67]:
t1, t2 = np.array([1,2,3]), np.array([12,13,14])
np.sqrt(np.mean((np.log(t1+1)-np.log(t2+1))**2)), travelTimeScore(t1, t2)
#travelEndScore(train_df['END'][0:4], train_df['END'][5:9])
c1, c2 = train_df['END'][0:4], train_df['END'][5:9]
#num_points = len(c1)
#num_points
#[c1[i] for i in range(num_points)]
s1 = travelEndScore(c1, c2)
s1

3.7391660042791557

In [81]:
def weightedMeanTravelTimes(train, test, num_trips=100):
    test['TRAVEL_TIME'] = 0
    for idx, start_coord in enumerate(test['START']):
        dists = train['START'].apply(lambda x: HaversineDistance(x, start_coord))
        smallest_dist_indexes = np.argpartition(dists, num_trips)[0:num_trips]
        w = np.maximum(dists.iloc[smallest_dist_indexes], 0.01)
        path_lengths = train.iloc[smallest_dist_indexes]['COORDS_LEN']
        no_path_lengths_indexes = np.argpartition(path_lengths, int(num_trips*.95))[0:int(num_trips*.95)]
        print idx
        #print "test[idx, travel_time]: ", test[idx, 'TRAVEL_TIME']
        print "test.loc(idx, coords_len): ", test.loc[idx, "COORDS_LEN"]
        print "avg: ", np.average(s.iloc[no_path_lengths_indexes], weights=1/w.iloc[no_path_lengths_indexes]**2)
        #test.loc[idx, 'TRAVEL_TIME'] = 15*np.maximum(test.loc[idx, 'COORDS_LEN'], np.average(s.iloc[no_path_lengths_indexes], weights=1/w.iloc[no_path_lengths_indexes]**2))
        test.loc[idx, 'TRAVEL_TIME'] = 15*np.maximum(test.loc[idx, 'COORDS_LEN'], np.average(path_lengths.iloc[no_path_lengths_indexes], weights=1/w.iloc[no_path_lengths_indexes]**2))

        
    test['TRAVEL_TIME'] = test['TRAVEL_TIME'].astype(int)
    return test['TRAVEL_TIME'].values


def weightedMeanTravelTimes3(train, test, num_trips=100):
    test_times = np.zeros(len(test))
    for idx, start_coord in enumerate(test['START'].values):
        dists = np.array([HaversineDistance(x, start_coord) for x in train['START'].values])
        smallest_dist_indexes = np.argpartition(dists, num_trips)[0:num_trips]
        w = np.maximum(dists[smallest_dist_indexes], 0.01)
        path_lengths = train['COORDS_LEN'].values[smallest_dist_indexes]
        no_path_lengths_indexes = np.argpartition(path_lengths, int(num_trips*.95))[0:int(num_trips*.95)]
        test_times[idx] = 15*np.maximum(test['COORDS_LEN'].values[idx], np.average(path_lengths[no_path_lengths_indexes], weights=1/w[no_path_lengths_indexes]**2))

        
    test['TRAVEL_TIME'] = test_times.astype(int)
    return test_times



def weightedMeanTravelTimes4(train, test, num_trips=100):
    test_times = np.zeros(len(test))
    train_lens = train['POLYLINE'].apply(len)
    for idx, test_coord in enumerate(test['POLYLINE'].values):
        dists_starts = np.array([HaversineDistance(x[0], test_coord[0]) for x in train['POLYLINE'].values])
        dists_ends = np.array([(HaversineDistance(x[(len(test_coord)-1)], test_coord[-1]) if len(x) > len(test_coord) else 100) for x in train['POLYLINE'].values ])
        dists = dists_starts + dists_ends
        smallest_dist_indexes = np.argpartition(dists, num_trips)[0:num_trips]
        w = np.maximum(dists[smallest_dist_indexes], 0.01)
        path_lengths = train['COORDS_LEN'].values[smallest_dist_indexes]
        no_path_lengths_indexes = np.argpartition(path_lengths, int(num_trips*.95))[0:int(num_trips*.95)]
        test_times[idx] = 15*np.maximum(test['COORDS_LEN'].values[idx], np.average(path_lengths[no_path_lengths_indexes], weights=1/w[no_path_lengths_indexes]**2))

        
    test['TRAVEL_TIME'] = test_times.astype(int)
    return test_times

In [111]:
num_records = 20000
taxi_df, submission_df = load_data(num_records_to_load=num_records)
#weightedMeanTravelTimes(taxi_df, submission_df, num_trips=10)

train_df, test_df, train_time, test_time, train_end, test_end = createTrainTestSplit(taxi_df, 400*n)
print "num training:", len(train_df), "   num test:", len(test_df)

#pred_time = weightedMeanTravelTimes4(train_df, test_df, 5)
#print "score:", travelTimeScore(pred_time, test_time)

validation_time = weightedMeanTravelTimes4(train_df, submission_df, 5)

#submit
submitTravelTime(submission_df, 'inverse_distance_kernel_2_points_B.csv')

num training: 17306    num test: 1933


/home/tony/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:42: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


In [300]:
pred_time = weightedMeanTravelTimes(train_df, test_df,10)



0
test.loc(idx, coords_len):  23
avg:  36.066944722
1
test.loc(idx, coords_len): 

KeyError: 'the label [1] is not in the [index]'